# Linear Regression

Based on must read book; [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/)

- [Load Datasets](#Load-Datasets)
- [3.1 Simple Linear Regression](#3.1-Simple-Linear-Regression)
- [3.2 Multiple Linear Regression](#3.2-Multiple-Linear-Regression)
- [3.3 Other Considerations in the Regression Model](#3.3-Other-Considerations-in-the-Regression-Model)

In [ ]:
# %load ../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns

from sklearn.preprocessing import scale
import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error, r2_score
 
%matplotlib inline
plt.style.use('seaborn-white')

## Bug fixes - statsmodels not compatible with current stable version of scipy/pandas

In [ ]:
# Statsmodels bug fix:
from pandas.core import datetools

import statsmodels.api as sm
import statsmodels.formula.api as smf

# Workaround to fix bug in statsmodels .summary() - missing stats.chisqprob function
# https://github.com/statsmodels/statsmodels/issues/3931
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

In [ ]:
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

### Load Datasets
Datasets available on http://www-bcf.usc.edu/~gareth/ISL/data.html

In [ ]:
! find ../../_data | grep -i advertising

In [ ]:
advertising = pd.read_csv('../../_data/Advertising.csv', usecols=[1,2,3,4])
advertising.info()

In [ ]:
credit = pd.read_csv('../../_data/Credit.csv', usecols=list(range(1,12)))
credit['Student2'] = credit.Student.map({'No':0, 'Yes':1})
credit.head(3)

In [ ]:
auto = pd.read_csv('../../_data/Auto.csv', na_values='?').dropna()
auto.info()

## 3.1 Simple Linear Regression

### Least squares fit - OLS

In [ ]:
sns.regplot(advertising.TV, advertising.Sales, order=1, ci=None, scatter_kws={'color':'r'})
plt.xlim(-10, 310)
plt.ylim(ymin=0);

### Regression coefficients - RSS
Note that the text in the book describes the coefficients based on uncentered data, whereas the plot shows the model based on centered data. The latter is visually more appealing for explaining the concept of a minimum RSS. I think that, in order not to confuse the reader, the values on the axis of the B0 coefficients have been changed to correspond with the text. The axes on the plots below are unaltered.

In [ ]:
# Regression coefficients (Ordinary Least Squares)
regr = skl_lm.LinearRegression()

X = scale(advertising.TV, with_mean=True, with_std=False).reshape(-1,1)
y = advertising.Sales

regr.fit(X, y)
print(regr.intercept_)
print(regr.coef_)

In [ ]:
# Create grid coordinates for plotting
B0 = np.linspace(regr.intercept_-2, regr.intercept_+2, 50)
B1 = np.linspace(regr.coef_-0.02, regr.coef_+0.02, 50)
bb, ww = np.meshgrid(B0, B1, indexing='xy')
Z = np.zeros((B0.size, B1.size))

# Grid RSS
# Calculate Z-values (RSS) based on grid of coefficients
for (i,j), v in np.ndenumerate(Z):
    Z[i,j] =((y - (X.ravel()*ww[i,j] + bb[i,j]))**2).sum()/1000

# Model RSS (minimised)
# min_rss = np.sum((regr.intercept_ + regr.coef_ * X - y.reshape(-1, 1))**2)/1000 # original with Future warning
min_rss = np.sum((regr.intercept_ + regr.coef_ * X - y.values.reshape(-1, 1))**2)/1000
'minimised RSS: ', min_rss

In [ ]:
fig = plt.figure(figsize=(18, 8))
fig.suptitle('RSS - Regression coefficients', fontsize=20)
legend = r'$\beta_0$, $\beta_1$ for minimized RSS'

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122, projection='3d')

# Countour plot (left)
CS = ax1.contour(xx, yy, Z, cmap=plt.cm.Set1, levels=[2.15, 2.2, 2.3, 2.5, 3])
ax1.clabel(CS, inline=True, fontsize=10, fmt='%1.2f')             # add contour values
ax1.scatter(regr.intercept_, regr.coef_[0], c='r', label=legend)  # plot min_RSS point


# 3D plot (right)
ax2.plot_surface(bb, ww, Z, rstride=3, cstride=3, alpha=0.3)                # plot spline/plane
ax2.contour(bb, ww, Z, zdir='z', offset=Z.min(), cmap=plt.cm.Set1,
            alpha=0.4, levels=[2.15, 2.2, 2.3, 2.5, 3])                     # plot contour
ax2.scatter3D(regr.intercept_, regr.coef_[0], min_rss, c='r', label=legend) # plot min_RSS point
ax2.set_zlabel('RSS')
ax2.set_zlim(Z.min(), Z.max())
ax2.set_ylim(0.02, 0.07)

# Settings common to both plots
for ax in fig.axes:
    ax.set_xlabel(r'$\beta_0$', fontsize=17)
    ax.set_ylabel(r'$\beta_1$', fontsize=17)
#     ax.set_yticks([0.03, 0.04, 0.05, 0.06])
    ax.legend()
plt.show();

### OLS - Coefficient table - Statsmodels (R like)

In [ ]:
import statsmodels.formula.api as smf

est = smf.ols('Sales ~ TV', advertising).fit()
est.summary().tables[1]

In [ ]:
# RSS with regression coefficients (y - (wx + b))^2
((advertising.Sales - (est.params[0] + est.params[1]*advertising.TV))**2).sum()/1000

In [ ]:
# Predict Sales 
est.predict(advertising.TV)[:5]

### Linear Regression - Scikit-learn

In [ ]:
import sklearn.linear_model as skl_lm

regr = skl_lm.LinearRegression()

X = advertising.TV.values.reshape(-1,1)
y = advertising.Sales

regr.fit(X, y)
print(r'β_0: {}, β_n: {}'.format(regr.intercept_, regr.coef_))

In [ ]:
Sales_pred = regr.predict(X)
r2_score(y, Sales_pred)

## 3.2 Multiple Linear Regression

### Statsmodels

In [ ]:
est = smf.ols('Sales ~ Radio', advertising).fit()
est.summary().tables[1]

In [ ]:
est = smf.ols('Sales ~ Newspaper', advertising).fit()
est.summary().tables[1]

In [ ]:
est = smf.ols('Sales ~ TV + Radio + Newspaper', advertising).fit()
est.summary()

### Correlation Matrix

In [ ]:
advertising.corr()

### Multiple Linear Regression

In [ ]:
X = advertising[['Radio', 'TV']].as_matrix()
y = advertising.Sales

regr = skl_lm.LinearRegression().fit(X, y)
print(r'β_0: {}, β_n: {}'.format(regr.intercept_, regr.coef_))

In [ ]:
# What are the min/max values of Radio & TV?
# Use these values to set up the grid for plotting.
advertising[['Radio', 'TV']].describe()

In [ ]:
# Create a coordinate grid - take max. feature value fro range
Radio = np.arange(50)
TV = np.arange(300)

B1, B2 = np.meshgrid(Radio, TV, indexing='xy')
Z = np.zeros((TV.size, Radio.size))

for (i,j),v in np.ndenumerate(Z):
        Z[i,j] =(regr.intercept_ + B1[i,j]*regr.coef_[0] + B2[i,j]*regr.coef_[1])

In [ ]:
# Create plot
fig = plt.figure(figsize=(10, 6))
fig.suptitle('Regression: Sales ~ Radio + TV Advertising', fontsize=20)

ax = axes3d.Axes3D(fig)

ax.plot_surface(B1, B2, Z, rstride=10, cstride=5, alpha=0.4)
ax.scatter3D(advertising.Radio, advertising.TV, advertising.Sales, c='r')

ax.set_xlabel('Radio')
ax.set_xlim(0,50)
ax.set_ylabel('TV')
ax.set_ylim(ymin=0)
ax.set_zlabel('Sales');

## 3.3 Other Considerations in the Regression Model

### Pairplot

In [ ]:
sns.pairplot(credit[['Balance','Age','Cards','Education','Income','Limit','Rating']]);

###  More linear statsmodels

In [ ]:
est = smf.ols('Balance ~ Gender', credit).fit()
est.summary().tables[1]

In [ ]:
est = smf.ols('Balance ~ Ethnicity', credit).fit()
est.summary().tables[1]

### Interaction Variables

In [ ]:
est = smf.ols('Sales ~ TV + Radio + TV*Radio', advertising).fit()
est.summary().tables[1]

### Interaction between qualitative and quantative variables

In [ ]:
est1 = smf.ols('Balance ~ Income + Student2', credit).fit()
regr1 = est1.params
est2 = smf.ols('Balance ~ Income + Income*Student2', credit).fit()
regr2 = est2.params

print('Regression 1 - without interaction term')
print(regr1)
print('\nRegression 2 - with interaction term')
print(regr2)

In [ ]:
# Income (x-axis)
income = np.linspace(0,150)

# Balance without interaction term (y-axis)
student1 = np.linspace(regr1['Intercept']+regr1['Student2'],
                       regr1['Intercept']+regr1['Student2']+150*regr1['Income'])
non_student1 =  np.linspace(regr1['Intercept'], regr1['Intercept']+150*regr1['Income'])

# Balance with iteraction term (y-axis)
student2 = np.linspace(regr2['Intercept']+regr2['Student2'],
                       regr2['Intercept']+regr2['Student2']+
                       150*(regr2['Income']+regr2['Income:Student2']))
non_student2 =  np.linspace(regr2['Intercept'], regr2['Intercept']+150*regr2['Income'])

# Create plot
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,5))
ax1.plot(income, student1, 'r', income, non_student1, 'k')
ax2.plot(income, student2, 'r', income, non_student2, 'k')

for ax in fig.axes:
    ax.legend(['student', 'non-student'], loc=2)
    ax.set_xlabel('Income')
    ax.set_ylabel('Balance')
    ax.set_ylim(ymax=1550)

plt.show();

### Non-linear relationships

In [ ]:
# With Seaborn's regplot() you can easily plot higher order polynomials.
plt.scatter(auto.horsepower, auto.mpg, facecolors='None', edgecolors='k') 
sns.regplot(auto.horsepower, auto.mpg, ci=None, label='Linear', scatter=False)
sns.regplot(auto.horsepower, auto.mpg, ci=None, label='Degree 2', order=2, scatter=False)
sns.regplot(auto.horsepower, auto.mpg, ci=None, label='Degree 5', order=5, scatter=False)
plt.legend()
plt.ylim(5,55)
plt.xlim(40,240);

### Table 3.10

In [ ]:
auto['horsepower2'] = auto.horsepower**2
auto.head(3)

In [ ]:
est = smf.ols('mpg ~ horsepower + horsepower2', auto).fit()
est.summary().tables[1]

### Figure 3.9

In [ ]:
# Ignore Future warnings about: y.reshape -> y.value.reshape
warnings.simplefilter('ignore', FutureWarning)

regr = skl_lm.LinearRegression()

# Linear fit
X = auto.horsepower.reshape(-1,1)
y = auto.mpg
regr.fit(X, y)

auto['pred1'] = regr.predict(X)
auto['resid1'] = auto.mpg - auto.pred1

# Quadratic fit
X2 = auto[['horsepower', 'horsepower2']].as_matrix()
regr.fit(X2, y)

auto['pred2'] = regr.predict(X2)
auto['resid2'] = auto.mpg - auto.pred2

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,5))

# Left plot
sns.regplot(auto.pred1, auto.resid1, lowess=True, 
            ax=ax1, line_kws={'color':'r', 'lw':1})
ax1.hlines(0,xmin=ax1.xaxis.get_data_interval()[0],
           xmax=ax1.xaxis.get_data_interval()[1], linestyles='dotted')
ax1.set_title('Residual Plot for Linear Fit')

# Right plot
sns.regplot(auto.pred2, auto.resid2, lowess=True,
            line_kws={'color':'r', 'lw':1}, ax=ax2)
ax2.hlines(0,xmin=ax2.xaxis.get_data_interval()[0],
           xmax=ax2.xaxis.get_data_interval()[1], linestyles='dotted')
ax2.set_title('Residual Plot for Quadratic Fit')

for ax in fig.axes:
    ax.set_xlabel('Fitted values')
    ax.set_ylabel('Residuals')
plt.show();

### Figure 3.14

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,5))

# Left plot
ax1.scatter(credit.Limit, credit.Age, facecolor='None', edgecolor='r')
ax1.set_ylabel('Age')

# Right plot
ax2.scatter(credit.Limit, credit.Rating, facecolor='None', edgecolor='r')
ax2.set_ylabel('Rating')

for ax in fig.axes:
    ax.set_xlabel('Limit')
    ax.set_xticks([2000, 4000, 6000, 8000, 12000])
plt.show();

### Figure 3.15

In [ ]:
y = credit.Balance

# Regression for left plot
X = credit[['Age', 'Limit']].as_matrix()
regr1 = skl_lm.LinearRegression()
regr1.fit(scale(X.astype('float'), with_std=False), y)

print('Age/Limit\n',regr1.intercept_)
print(regr1.coef_)

# Regression for right plot
X2 = credit[['Rating', 'Limit']].as_matrix()
regr2 = skl_lm.LinearRegression()
regr2.fit(scale(X2.astype('float'), with_std=False), y)

print('\nRating/Limit\n',regr2.intercept_)
print(regr2.coef_)

In [ ]:
# Create grid coordinates for plotting
B_Age = np.linspace(regr1.coef_[0]-3, regr1.coef_[0]+3, 100)
B_Limit = np.linspace(regr1.coef_[1]-0.02, regr1.coef_[1]+0.02, 100)

B_Rating = np.linspace(regr2.coef_[0]-3, regr2.coef_[0]+3, 100)
B_Limit2 = np.linspace(regr2.coef_[1]-0.2, regr2.coef_[1]+0.2, 100)

X1, Y1 = np.meshgrid(B_Limit, B_Age, indexing='xy')
X2, Y2 = np.meshgrid(B_Limit2, B_Rating, indexing='xy')
Z1 = np.zeros((B_Age.size,B_Limit.size))
Z2 = np.zeros((B_Rating.size,B_Limit2.size))

Limit_scaled = scale(credit.Limit.astype('float'), with_std=False)
Age_scaled = scale(credit.Age.astype('float'), with_std=False)
Rating_scaled = scale(credit.Rating.astype('float'), with_std=False)

# Calculate Z-values (RSS) based on grid of coefficients
for (i,j),v in np.ndenumerate(Z1):
    Z1[i,j] =((y - (regr1.intercept_ + X1[i,j]*Limit_scaled +
                    Y1[i,j]*Age_scaled))**2).sum()/1000000
    
for (i,j),v in np.ndenumerate(Z2):
    Z2[i,j] =((y - (regr2.intercept_ + X2[i,j]*Limit_scaled +
                    Y2[i,j]*Rating_scaled))**2).sum()/1000000

In [ ]:
fig = plt.figure(figsize=(12,5))
fig.suptitle('RSS - Regression coefficients', fontsize=20)

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

min_RSS = r'$\beta_0$, $\beta_1$ for minimized RSS'
    
# Left plot
CS = ax1.contour(X1, Y1, Z1, cmap=plt.cm.Set1, levels=[21.25, 21.5, 21.8])
ax1.scatter(regr1.coef_[1], regr1.coef_[0], c='r', label=min_RSS)
ax1.clabel(CS, inline=True, fontsize=10, fmt='%1.1f')
ax1.set_ylabel(r'$\beta_{Age}$', fontsize=17)

# Right plot
CS = ax2.contour(X2, Y2, Z2, cmap=plt.cm.Set1, levels=[21.5, 21.8])
ax2.scatter(regr2.coef_[1], regr2.coef_[0], c='r', label=min_RSS)
ax2.clabel(CS, inline=True, fontsize=10, fmt='%1.1f')
ax2.set_ylabel(r'$\beta_{Rating}$', fontsize=17)
ax2.set_xticks([-0.1, 0, 0.1, 0.2])

for ax in fig.axes:
    ax.set_xlabel(r'$\beta_{Limit}$', fontsize=17)
    ax.legend()
plt.show();

### Variance Inflation Factor - page 102

In [ ]:
est_Age = smf.ols('Age ~ Rating + Limit', credit).fit()
est_Rating = smf.ols('Rating ~ Age + Limit', credit).fit()
est_Limit = smf.ols('Limit ~ Age + Rating', credit).fit()

print(1/(1-est_Age.rsquared))
print(1/(1-est_Rating.rsquared))
print(1/(1-est_Limit.rsquared))